# Setting connection params

Please adjust IP and user/pass if you are connecting to anything other than singlehost. Or if your singlehost has another IP.

In [128]:
singlehost="192.168.10.11"
buildbox="192.168.10.12"
apibox="192.168.10.13"
molo_host = singlehost
ela_host = singlehost

In [129]:
user = "vagrant"
passwd = "vagrant"

In [130]:
molo_conn_string = "http://{}:8005".format(molo_host)
molo_conn_string

'http://192.168.10.11:8005'

In [131]:
ela_conn_string = "http://{}:9200".format(ela_host)

In [132]:
class Get():
    CONN_JSON      = "connections.json"
    CONN_CSV       = "connections.csv"
    FILES          = "file/list"
    SESS_JSON      = "sessions.json"
    SESS_CSV       = "sessions.csv"
    SESS_PCAP      = "sessions.pcap"
    SPI_GRAPH_JSON = "spigraph.json"
    SPI_VIEW_JSON  = "spiview.json"
    UNIQ           = "unique.txt"
    
class Post():
    TAGS_ADD       = "addTags"
    TAGS_REM       = "removeTags"

# Listing available fields in moloch database

In [133]:
from elasticsearch import Elasticsearch

In [134]:
es = Elasticsearch(hosts=[ela_conn_string])
es.ping()

True

In [135]:
fields = es.search("fields", params={"size":3000})
fields = [f["_source"] for f in fields["hits"]["hits"]]
fields = [f["dbField2"] for f in fields]
fields = sorted(fields)
fields

['_id',
 'asnall',
 'asset',
 'assetCnt',
 'cdmcs.name',
 'cdmcs.nameCnt',
 'cdmcs.type',
 'cdmcs.typeCnt',
 'cert.alt',
 'cert.altCnt',
 'cert.hash',
 'cert.issuerCN',
 'cert.issuerON',
 'cert.notAfter',
 'cert.notBefore',
 'cert.remainingDays',
 'cert.serial',
 'cert.subjectCN',
 'cert.subjectON',
 'cert.validDays',
 'certCnt',
 'communityId',
 'dhcp.host',
 'dhcp.hostCnt',
 'dhcp.hostTokens',
 'dhcp.id',
 'dhcp.idCnt',
 'dhcp.mac',
 'dhcp.macCnt',
 'dhcp.oui',
 'dhcp.ouiCnt',
 'dhcp.type',
 'dhcp.typeCnt',
 'dns.ASN',
 'dns.GEO',
 'dns.RIR',
 'dns.host',
 'dns.hostCnt',
 'dns.hostTokens',
 'dns.ip',
 'dns.ipCnt',
 'dns.mailserverASN',
 'dns.mailserverGEO',
 'dns.mailserverHost',
 'dns.mailserverHostCnt',
 'dns.mailserverIp',
 'dns.mailserverIpCnt',
 'dns.mailserverRIR',
 'dns.nameserverASN',
 'dns.nameserverGEO',
 'dns.nameserverHost',
 'dns.nameserverHostCnt',
 'dns.nameserverIp',
 'dns.nameserverIpCnt',
 'dns.nameserverRIR',
 'dns.opcode',
 'dns.opcodeCnt',
 'dns.puny',
 'dns.puny

# Talking to endpoints

In [136]:
import requests
from requests.auth import HTTPDigestAuth

Consider the following example:

```
curl -u vagrant:vagrant --digest \
    --data-urlencode "startTime=$(date -d '04/16/2019 17:00:00' +"%s")" \
    --data-urlencode "stopTime=$(date -d '04/16/2019 18:00:00' +"%s")" \
    --data-urlencode "expression=protocols==dns" \
    --data-urlencode "exp=dns.host" \
    -GET "http://192.168.10.11:8005/unique.txt"
```

Firstly, we can make the unix epoch parsing a bit more convenient. Following datetime lines are equal to unix *date* command output.

In [137]:
import datetime as dt
start = int(dt.datetime.strptime('04/16/2019 17:00:00', '%m/%d/%Y %H:%M:%S').strftime("%s"))
end = int(dt.datetime.strptime('04/16/2019 18:00:00', '%m/%d/%Y %H:%M:%S').strftime("%s"))

We can wrap this in a simple function. Feel free to use any date format that is convenient to you.

In [138]:
def myDate(stringdate):
    return int(dt.datetime.strptime(stringdate, '%m/%d/%Y %H:%M:%S').strftime("%s"))

Dividing the end from start does indeed result in 3600 seconds, or 1 hour.

In [139]:
myDate('04/16/2019 18:00:00') - start

3600

Now, let's construct the body of the request.

In [140]:
query = {
    "startTime": start,
    "stopTime": end,
    "expression": "protocols == dns",
    "exp": "dns.host"
}

And send out the digest authenticated GET request. Collect results into a variable.

In [141]:
endpoint = "{}/{}".format(molo_conn_string, Get.UNIQ)
endpoint

'http://192.168.10.11:8005/unique.txt'

In [142]:
resp = requests.get(endpoint, params=query, auth=HTTPDigestAuth(user, passwd))
resp

<Response [200]>

In [143]:
resp.text

''

Note the okay response code 200 with empty string in results. If using singlehost. That's because you are unlikely to have any data from example period. As an exercise, lets override this with last hour. This is equivelant to `date=1`, but does not matter for exercise.

In [144]:
query["startTime"] = int(dt.datetime.now().strftime("%s"))-3600
query["stopTime"] = dt.datetime.now().strftime("%s")

In [145]:
resp = requests.get(endpoint, params=query, auth=HTTPDigestAuth(user, passwd))
resp

<Response [200]>

In [146]:
resp.text

'berylia.org\nstar-mini.c10r.facebook.com\ngithub.map.fastly.net\nsysadminnid.tumblr.com\nself-signed.badssl.com\ntestmyids.com\nraw.githubusercontent.com\ngrafana.com\nwww.facebook.com\n15.2.0.10.in-addr.arpa\n3.2.0.10.in-addr.arpa\ntestmyids.com.berylia.org\n'

Unique api returns a newline delimited string. Let's turn it into something more structured with basic python string functions. And get rid of empty elements left over by splitting logic.

In [147]:
domains = resp.text.split("\n")
print(domains)
domains = [d for d in domains if d != ""]
print(domains)

['berylia.org', 'star-mini.c10r.facebook.com', 'github.map.fastly.net', 'sysadminnid.tumblr.com', 'self-signed.badssl.com', 'testmyids.com', 'raw.githubusercontent.com', 'grafana.com', 'www.facebook.com', '15.2.0.10.in-addr.arpa', '3.2.0.10.in-addr.arpa', 'testmyids.com.berylia.org', '']
['berylia.org', 'star-mini.c10r.facebook.com', 'github.map.fastly.net', 'sysadminnid.tumblr.com', 'self-signed.badssl.com', 'testmyids.com', 'raw.githubusercontent.com', 'grafana.com', 'www.facebook.com', '15.2.0.10.in-addr.arpa', '3.2.0.10.in-addr.arpa', 'testmyids.com.berylia.org']


Generating new queries from the result of the first is now trivial.

In [197]:
endpoint = "{}/{}".format(molo_conn_string, Get.SESS_CSV)
resps = {}
for d in domains:
    r = requests.get(endpoint, params={
        "date": 2,
        "expression": "host.dns == {}".format(d),
        "fields": ",".join([
            "srcIp",
            "dstIp",
            "communityId",
            "dns.status"
        ])
    }, auth=HTTPDigestAuth(user, passwd))
    resps[d] = r.text

In [198]:
for k, v in resps.items():
    print("-----{}-----".format(k))
    print(v)

-----berylia.org-----
Src IP, Dst IP, Community Id, Status Code
10.0.2.15,1.1.1.1,1:tagj3MtXCUoAV2S+4dv7wgCAdXk=,"NOERROR"
10.0.2.15,1.1.1.1,1:3NWJhYyXZT2f1IORDdgDZ9crAaE=,"NOERROR"
10.0.2.15,1.1.1.1,1:rGRhDccXRNL0G/Ak3CObIAucHhQ=,"NOERROR"
10.0.2.15,8.8.8.8,1:WDVmkWWetGZGHLz+YMn4ljnb4yI=,"NOERROR"
10.0.2.15,1.1.1.1,1:eMLUZD41lqJHeqlqxl7DYeQLX+Q=,"NOERROR"
10.0.2.15,8.8.8.8,1:mxocn6g3dkmByhRhzpmUfOAAW9U=,"NOERROR"
10.0.2.15,8.8.8.8,1:LlozqkA19zX9r7bFxFwtojyUnoc=,"NOERROR"
10.0.2.15,1.1.1.1,1:R+iWVwLCc1MAkS1bHzRC38SCCNk=,"NOERROR"
10.0.2.15,8.8.8.8,1:esp1UznZdqQjTiIbsrX3GBE/C0M=,"NOERROR"
10.0.2.15,1.1.1.1,1:aNiWJmt8Lh25Z6d+qSKIZwYIMnc=,"NOERROR"
10.0.2.15,8.8.8.8,1:IgHj10AtyJMVBsA3MAo3KDCdz4c=,"NOERROR"
10.0.2.15,1.1.1.1,1:DtpF1/mHEs/o0TSlqGOaj43/TY4=,"NOERROR"
10.0.2.15,8.8.8.8,1:z3Cn6xv3s4jlLRaGZUG8KjFjtl0=,"NOERROR"
10.0.2.15,1.1.1.1,1:Kfk7dqbOCNgzuaYLKdiN1RhaHic=,"NOERROR"
10.0.2.15,8.8.8.8,1:JseIiyi9s3dblq52MO4hJCEiVFA=,"NOERROR"
10.0.2.15,1.1.1.1,1:QWjZtnqRS3PJgCBaBCFNMES8Vgs=,"N

Note the **fields** key in query. Since we know all fields and are doing queries programmatically, we could simply extract all dns fields from the list and ask them from moloch.

In [ ]:
dnsfields = [f for f in fields if "dns." in f]

In [200]:
endpoint = "{}/{}".format(molo_conn_string, Get.SESS_CSV)
resps = {}
for d in domains:
    r = requests.get(endpoint, params={
        "date": 2,
        "expression": "host.dns == {}".format(d),
        "fields": ",".join(dnsfields)
    }, auth=HTTPDigestAuth(user, passwd))
    resps[d] = r.text

In [203]:
for k, v in resps.items():
    print("-----{}-----".format(k))
    print(v.split("\n")[1])

-----berylia.org-----
,,,"berylia.org",1,,,,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"NS",1,"NOERROR",1
-----star-mini.c10r.facebook.com-----
"AS32934 Facebook, Inc.","IE","RIPE","star-mini.c10r.facebook.com",1,,"31.13.72.36",1,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"A",1,"NOERROR",1
-----github.map.fastly.net-----
,,,"raw.githubusercontent.com, github.map.fastly.net",2,,,,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"AAAA",1,"NOERROR",1
-----sysadminnid.tumblr.com-----
,,,"sysadminnid.tumblr.com",1,,,,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"AAAA",1,"NOERROR",1
-----self-signed.badssl.com-----
,,,"self-signed.badssl.com",1,,,,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"AAAA",1,"NOERROR",1
-----testmyids.com-----
,,,"testmyids.com",1,,,,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"AAAA",1,"NOERROR",1
-----raw.githubusercontent.com-----
,,,"raw.githubusercontent.com, github.map.fastly.net",2,,,,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"AAAA",1,"NOERROR",1
-----grafana.com-----
,,,"grafana.com",1,,,,,,,,,,,,,,,,,,"QUERY",1,,,"IN",1,"AAAA",1,"NOERRO

Some fields are missing because we are only looking at **query**, but not **response**.

# Tasks

A little dry run before going on a hunting trip...

## Basic

 * See the [bash examples](https://github.com/ccdcoe/CDMCS/tree/master/Moloch/queries#bash-examples)
     * Implement user-agent and ja3 example as python scripts
     
## Advanced
 
 * See the [bash examples](https://github.com/ccdcoe/CDMCS/tree/master/Moloch/queries#bash-examples) and [suricata eve json parsing notebook](https://github.com/ccdcoe/CDMCS/blob/master/Suricata/indexing/001-load-eve.ipynb). 
     * Your singlehost should have suricata alert file in **/var/log/suricata/eve.json**.
     * Copy that file to api box and run moloch session queries for all community ID values